# Simulation 7: Quantum Foam Stability with φ-Modulated Hamiltonian

**Goal**: Simulate how quantum foam (random spacetime fluctuations) perturbs a two-qubit entangled system, and determine if φ-modulation (using the golden ratio) improves entanglement stability by reducing variance in concurrence.

This simulation compares a base Hamiltonian with full-strength foam vs. a φ-enhanced version with higher base frequency and attenuated noise.

**Key Metrics:**
- Von Neumann Entropy $S = -\mathrm{Tr}(\rho \log \rho)$ (measures coherence)
- Concurrence $C(\rho)$ (measures entanglement)
- Variance in Concurrence (lower = more stable under foam)

In [ ]:
# Imports
import qutip as qt
import numpy as np
from numpy import sqrt, log
import matplotlib.pyplot as plt

## Define helper functions

In [ ]:
def von_neumann_entropy(rho):
    evals = np.linalg.eigvalsh(rho.full())
    evals = evals[evals > 1e-10]  # Avoid log(0)
    return -np.sum(evals * np.log(evals))

## Set simulation constants

In [ ]:
N = 2  # 2-qubit system
gamma = 0.1  # Decoherence rate
omega = 1.0  # Base frequency
phi = (1 + np.sqrt(5)) / 2  # Golden ratio
tlist = np.linspace(0, 10, 100)  # Time evolution
np.random.seed(42)  # Reproducibility

## Construct base Hamiltonian + foam perturbations

In [ ]:
# Base H: ω * (σz ⊗ I + I ⊗ σz) / 2
H_base = omega * (qt.tensor(qt.sigmaz(), qt.qeye(2)) + qt.tensor(qt.qeye(2), qt.sigmaz())) / 2

# Foam: Random σx/σy terms with small magnitude (ε ≈ 0.01)
foam_ops = [
    np.random.randn() * 0.01 * qt.tensor(qt.sigmax(), qt.qeye(2)),
    np.random.randn() * 0.01 * qt.tensor(qt.qeye(2), qt.sigmax()),
    np.random.randn() * 0.01 * qt.tensor(qt.sigmay(), qt.qeye(2)),
    np.random.randn() * 0.01 * qt.tensor(qt.qeye(2), qt.sigmay())
]

H_foam_base = H_base + sum(foam_ops)

## Define collapse operators (decoherence)

In [ ]:
c_ops = [
    sqrt(gamma) * qt.tensor(qt.sigmam(), qt.qeye(2)),
    sqrt(gamma) * qt.tensor(qt.qeye(2), qt.sigmam())
]

## Initial state: Bell state (max entangled)

In [ ]:
psi0 = (qt.tensor(qt.basis(2, 0), qt.basis(2, 0)) + qt.tensor(qt.basis(2, 1), qt.basis(2, 1))).unit()

## Run base simulation

In [ ]:
result_base = qt.mesolve(H_foam_base, psi0, tlist, c_ops)
ent_base = [von_neumann_entropy(state) for state in result_base.states]
conc_base = [qt.concurrence(state) for state in result_base.states]

## Create φ-modulated version (resonant boost and foam damping)

In [ ]:
H_phi = omega * phi * (qt.tensor(qt.sigmaz(), qt.qeye(2)) + qt.tensor(qt.qeye(2), qt.sigmaz())) / 2
foam_ops_phi = [f * (1 / phi) for f in foam_ops]  # Dampen foam
H_foam_phi = H_phi + sum(foam_ops_phi)
result_phi = qt.mesolve(H_foam_phi, psi0, tlist, c_ops)
ent_phi = [von_neumann_entropy(state) for state in result_phi.states]
conc_phi = [qt.concurrence(state) for state in result_phi.states]

## Plotting and Summary

In [ ]:
plt.plot(tlist, conc_base, label='Base')
plt.plot(tlist, conc_phi, label='φ-Modulated')
plt.title('Concurrence over Time')
plt.xlabel('Time')
plt.ylabel('Concurrence')
plt.legend()
plt.grid(True)
plt.show()

print("Base Concurrence Variance:", np.var(conc_base))
print("φ-Modulated Concurrence Variance:", np.var(conc_phi))